# Build a Prompt Chaining Agentic Workflow Using CrewAI

In this lab, we will build a chained prompts workflow using CrewAI Agentic Framework to intelligently process a legal contract, draft an email with concerns in the contract, and finally provide feedback on the email. Chaining prompts, which involves using the output of one prompt as input to another, offers several advantages when working with Large Language Models (LLMs).

## What is Prompt Chaining
Prompt chaining is a technique that involves breaking down a workflow into a series of known steps. Then orchestrates each step (typically involve an LLM invocation) to handle the output generated from the previous step. Within the worklow, there could be one or more conditional steps which determines the the trajectory of the workflow based on the given state.

## Architecture Diagram
<img src="../../imgs/prompt-chaining-architecture.png" width=800>

Let's get started!

## Import Required Libraries
We start by importing the necessary libraries for our prompt chaining workflow:
Flow, listen, start from CrewAI for building the workflow
BaseModel from Pydantic for data modeling
LLM from CrewAI for language model interactions

In [ ]:
from crewai.flow.flow import Flow, listen, start
from pydantic import BaseModel
from crewai import LLM

## Configure Logging
Set up logging configuration to track the execution of our workflow. This will help us monitor the progress and debug any issues that might arise during the contract analysis process.

In [ ]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

Define helper functions

In [ ]:
import boto3
from botocore.exceptions import ClientError

def retrieve_from_s3(bucket_name, contract_name):
    """
    Retrieve a contract document from S3
    
    Args:
        bucket_name (str): Name of the S3 bucket
        contract_name (str): Key/filename of the contract in S3
    
    Returns:
        str: Content of the contract document
    """
    try:
        # Create S3 client
        s3_client = boto3.client('s3')
        
        # Get the object from S3
        response = s3_client.get_object(Bucket=bucket_name, Key=contract_name)
        
        # Read and decode the content
        contract_content = response['Body'].read().decode('utf-8')
        
        return contract_content
        
    except ClientError as e:
        logger.error(f"Error retrieving contract from S3: {e}")
        return None
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        return None

## Define Agent State and LLM Configuration
Create the data model that will hold the state throughout our workflow. The AgentState class tracks:

- S3 bucket and object information for the contract
- Email concerns identified from the contract
- Generated email content
- Email review feedback
- We also configure the LLM (e.g. Amazon Nova Micro) and set the default S3 bucket for our lab.


In [ ]:
import boto3 

class AgentState(BaseModel):
    s3_bucket_name: str = None
    s3_object_key: str = None
    email_concerns: str = ""
    email_content: str = ""
    email_review_feedback: str = ""

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
    
llm = LLM(model="bedrock/us.amazon.nova-micro-v1:0")
default_bucket_name = f"labs-bucket-{region}-{account_id}" # replace the bucket name if running outside of the AWS facilitated environment.

## Contract Analysis Workflow Implementation
This is the core of our prompt chaining workflow. The ContractAnalysisFlow class defines a sequential flow with four stages:

1. Concern Finder: Acts as a Chief Legal Officer to identify risks in the contract (data privacy, SLAs, liability caps)
2. Email Writer: Drafts a professional email to outline the identified concerns
3. Email Reviewer: Reviews the drafted email and provides feedback on tone, clarity, and professionalism
4. Email Rewriter: Incorporates the feedback to produce a final, improved version of the email

Each step uses the @listen decorator to ensure proper sequencing, with each step waiting for the previous one to complete.


In [ ]:
class ContractAnalysisFlow(Flow[AgentState]):

    @start()
    def concern_finder(self):
        system_prompt = """You are our Chief Legal Officer. You are given a contract to review for risks, focusing on data privacy, SLAs, and liability caps.

Output your findings in <risks> XML tags.
"""
        contract_s3_key = self.state.s3_object_key
        bucket_name = self.state.s3_bucket_name
        contract_data = retrieve_from_s3(bucket_name, contract_s3_key)
        formatted_contract_data = f"<contract>\n{contract_data}\n</contract>"
    
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": formatted_contract_data}
        ]
        
        response = llm.call(messages=messages)
        self.state.email_concerns = response
        
    @listen(concern_finder)
    def email_writer(self):
        system_prompt = """You are an expert at writing corporate legal emails. You are given the concerns from a contract in <risk> XML tag. Your task is to draft an email to Frost Technologies outlining the following concerns and proposing changes to a contract.
"""
        concerns = self.state.email_concerns
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": concerns}
        ]
        
        response = llm.call(messages=messages)
        self.state.email_content = response
        
    @listen(email_writer)
    def email_reviewer(self):
        system_prompt = """You are a professional email reviewer. You are given an email content in <email> XML tag. Your task is to review an email and provide feedback. 
Give feedback on tone, clarity, and professionalism.
"""     
        email_content = self.state.email_content
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"<email>{email_content}</email>"}
        ]
        response = llm.call(messages=messages)
        self.state.email_review_feedback = response
        
    
    @listen(email_reviewer)
    def email_rewriter(self):
        system_prompt = """You are a professional email writer. You are given an original email content in <email> XML tag, and the feedback from a reviewer in <feedback> XML tag. Your task is to incorporate the feedback and rewrite the email from the orignial email.
Put your rewritten email in the <rewritten_email> XML tag. Do not provide any explainations.
"""     
        review_feedback = self.state.email_review_feedback
        email_content = self.state.email_content
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"<email>{email_content}</email>\n<feedback>{review_feedback}</feedback>"}
        ]
        response = llm.call(messages=messages) 
        return { "rewritten_email" : response }


## Execute the Workflow
Run the contract analysis workflow by providing the S3 bucket and contract file location. The workflow will execute asynchronously through all four stages and return the final result containing the rewritten email.

In [ ]:
response = await ContractAnalysisFlow().kickoff_async(inputs = {
    "s3_object_key" : "contract.txt",
    "s3_bucket_name" : default_bucket_name })
print(response)

## Summary
This notebook demonstrates a complete prompt chaining workflow using CrewAI for contract analysis and email generation. The workflow showcases several key concepts:

### Key Components:
- Sequential Processing: Each step builds upon the output of the previous step
- Specialized Roles: Different LLM personas (Legal Officer, Email Writer, Reviewer, Rewriter)
- State Management: Persistent state tracking across all workflow steps

### Workflow Stages:
- Contract Risk Analysis: Identifies legal concerns in contracts
- Email Drafting: Creates professional correspondence based on identified risks
- Quality Review: Evaluates email quality and professionalism
- Content Refinement: Produces final polished communication